In [ ]:
import time as ttime

from bluesky.plans import count, scan
from ophyd import EpicsSignal

Here the ophyd areadetector classes are setup:

In [ ]:
from ophyd.areadetector.filestore_mixins import FileStoreHDF5IterativeWrite
from ophyd.areadetector.plugins import HDF5Plugin_V34
from ophyd import SingleTrigger, AreaDetector, ADComponent
from ophyd.areadetector import cam

class MyHDF5Plugin(FileStoreHDF5IterativeWrite, HDF5Plugin_V34):
    ...


class MyDetector(SingleTrigger, AreaDetector):
    cam = ADComponent(cam.AreaDetectorCam, "CAM:")
    hdf1 = ADComponent(
        MyHDF5Plugin,
        "HDF1:",
        write_path_template="/out/%Y/%m/%d/",
        #read_path_template="/home/bar/Projects/tomoscan/data/%Y/%m/%d/",
        read_path_template="/home/jovyan/data/%Y/%m/%d/"
        
    )

In [ ]:
from ophyd import Device, Component, EpicsSignalRO

class MyLaser(Device):
    power = Component(EpicsSignalRO, "laser:power")
    pulse_id = Component(EpicsSignalRO, "EPAC-DEV:PULSE:PULSE_ID", name="pulse_id")

The next code block defines a function used to poll PVs until they take a particular value.
This function should be a temporary measure and a more streamlined approach monitoring the PV integrated proerly with Bluesky/Ophyd is hoped to be used in the future.

In [ ]:
def wait_for_value(signal: EpicsSignal, value, poll_time=0.01, timeout=10):
    expiration_time = ttime.time() + timeout
    current_value = signal.get()
    while current_value != value:
        # ttime.sleep(poll_time)
        yield from bps.sleep(poll_time)
        if ttime.time() > expiration_time:
            raise TimeoutError(
                "Timed out waiting for %r to take value %r after %r seconds"
                % (signal, value, timeout)
            )
        current_value = signal.get()

This custom plan moves the motor and then waits for the laser pulse before taking the next reading.

In [ ]:
from bluesky.plan_stubs import mv
import bluesky.plan_stubs as bps

def pulse_sync(detectors, motor, laser, start, stop, steps):
    step_size = (stop - start) / (steps - 1)

    for det in detectors:
        yield from bps.stage(det)

    yield from bps.open_run()
    for i in range(steps):
        yield from bps.checkpoint()  # allows pausing/rewinding
        yield from mv(motor, start + i * step_size)
        yield from wait_for_value(
            laser.power, 0, poll_time=0.01, timeout=10
        )  # Want to be at 0 initially such that image taken on pulse
        yield from wait_for_value(laser.power, 1, poll_time=0.001, timeout=10)
        yield from bps.trigger_and_read(list(detectors) + [motor] + [laser])
    yield from bps.close_run()

    for det in detectors:
        yield from bps.unstage(det)

This custom plan moves the motor based on the detector status. It is designed to be used when the detector is being directly triggered outside of bluesky.

In [ ]:
def passive_scan(detectors, motor, start, stop, steps, adStatus, pulse_ID):
    step_size = (stop - start) / (steps - 1)

    yield from mv(motor, start)  # Move motor to starting position since may take time

    yield from bps.open_run()

    for det in detectors:
        yield from bps.stage(det)

    for i in range(steps):
        yield from mv(motor, start + i * step_size)
        yield from bps.checkpoint()
        yield from wait_for_value(adStatus, 2, poll_time=0.001, timeout=10)
        yield from bps.trigger_and_read([motor] + [pulse_ID])
        yield from wait_for_value(adStatus, 0, poll_time=0.001, timeout=10)

    for det in detectors:
        yield from bps.unstage(det)

    yield from bps.close_run()

Here we initialise the detector object. 
This will give an error saying that caRepeater couldn't be located, this is not an issue. A second related error message may later appear which can also be ignored.

In [ ]:
prefix = "ADT:USER1:"
det = MyDetector(prefix, name="det")

The detector's HDF plugin is then set to create the necessary output directory if it does not exist and the detector is primed.
The plugin's "kind" is required to be set to 3 such that the resulting HDF files are accessible via the databroker.

In [ ]:
det.hdf1.create_directory.put(-5)
det.hdf1.warmup()
det.hdf1.kind = 3  # config | normal, required to include images in run documents

The camera's stage signatures are area detector configurations which are set whenever the detector is staged within Bluesky.

In [ ]:
det.cam.stage_sigs["image_mode"] = "Multiple"
det.cam.stage_sigs["acquire_time"] = 0.05
det.cam.stage_sigs["num_images"] = 1

The motor and laser objects are created. In the case of the laser we wait for the PVs to connect succesfully.

In [ ]:
from ophyd import EpicsMotor

motor1 = EpicsMotor("motorS:axis1", name="motor1")
laser1 = MyLaser("", name="laser1")
laser1.wait_for_connection()

We load the Bluesky run engine and subscribe the best effort callback. The best effort callback aims to print and plot useful information as scans are performed.

In [ ]:
from bluesky import RunEngine
from bluesky.callbacks.best_effort import BestEffortCallback

RE = RunEngine()
bec = BestEffortCallback()
RE.subscribe(bec)

The databroker is linked to the running mongoDB database and the run engine is set to insert all data captured to the databroker.

In [ ]:
import databroker

catalog = databroker.catalog["mongo"]
RE.subscribe(catalog.v1.insert)

As a first example run this synced scan which takes 11 readings at intervals between motor positions of -10 and +10
This will also generate a table and a plot of the motor position. In this case the plot does not show much information.

In [ ]:
uids = RE(pulse_sync([det], motor1, laser1, -10, 10, 11))

To access the data stored in the databroker catalog run the following:

In [ ]:
run = catalog[uids[0]] #Accesses the run based on its uid, the most recent run can also be accessed as catalog[-1]
data = run.primary.read()
data

The image data is accessed as shown below.

In [ ]:
image = data["det_image"]
frame = image[0][0]  # Index 1 refers to the time of the image and the second index refers to the frame number
frame.plot.pcolormesh()